In [5]:
import pandas as pd
import numpy as np

def gen_winners(player, path):
    # Load the data
    events = pd.read_csv(path)
    events['pointWonBy'] = events.groupby('pointNumber')['pointWonBy'].bfill()
    events['isVolley'] = events['isVolley'].fillna(0)

    # Filter for the player's returns and shots in rally
    winners = events[(events['shotHitBy'] == player) & (events['isWinner'] == 1.0) & (events['pointWonBy'] == player)][['shotHitBy', 'pointStartTime', 'shotContactX', 'shotContactY', 'shotLocationX', 'shotLocationY', 'pointWonBy', 'isVolley', 'shotFhBh']].dropna(subset=['pointWonBy']).copy()

    # Adjust shotLocationY and shotLocationX and flip shotContactX, shotContactY when shotLocation > 0
    winners = winners.dropna()
    winners['shotLocationX'] = winners.apply(lambda row: -row['shotLocationX'] if row['shotLocationY'] < 0 else row['shotLocationX'], axis=1)
    winners['shotContactX'] = winners.apply(lambda row: -row['shotContactX'] if row['shotLocationY'] < 0 else row['shotContactX'], axis=1)
    winners['shotContactY'] = winners.apply(lambda row: -row['shotContactY'] if row['shotLocationY'] < 0 else row['shotContactY'], axis=1)
    winners['shotLocationY'] = winners.apply(lambda row: -row['shotLocationY'] if row['shotLocationY'] < 0 else row['shotLocationY'], axis=1)

    # Export the data as JSON
    winners_json = winners.to_json(orient='records')

    with open('winners.json', 'w') as f:
        f.write(winners_json)


player = "Anne Lutkemeyer"
path = "../Match CSVs/Shot_Visuals_AnneLutkemeyer_CatherineGagnon.csv"
gen_winners(player, path)